In [1]:
import pandas as pd
import re
import os
import numpy as np
import glob
from datetime import datetime
import geopandas as gpd
import json
from shapely.geometry import shape, GeometryCollection
from rasterio.plot import show
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors
from utils import FeatureExtraction, feature_matching, decimal_coords
from exif import Image as exifimg
from PIL import Image

import shutil
import rasterio
import cv2
%matplotlib inline

In [2]:
# 가능한 font list 확인
import matplotlib.font_manager as fm
f = [f.name for f in fm.fontManager.ttflist]
# print(f)

plt.rc('font', family='Malgun Gothic')

In [3]:
import boto3
from decimal import Decimal
# Get the service resource.
dynamodb = boto3.resource('dynamodb', region_name='ap-northeast-2')
table = dynamodb.Table('dsnia_imagelabelsV2')
# table = dynamodb.Table('dsnia_imagelabels')

In [4]:
AWS_folder = os.getcwd()

b_folder = os.path.join(AWS_folder,'BACKUPS')
b_folder

src_folder = os.path.join(AWS_folder,'dataDownloaded')
src_folder

out_folder = os.path.join(AWS_folder,'filepathBackUp')
out_folder

'c:\\Users\\yc463\\Documents\\JamesLocal\\DS\\Drone\\dsmapv2\\src\\AWS\\filepathBackUp'

In [5]:
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError



response = table.scan()
alldata = response['Items']

while 'LastEvaluatedKey' in response:
    response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    alldata.extend(response['Items'])

In [6]:
len(alldata)

176305

In [7]:

from datetime import datetime
 
datetime.now().strftime("%Y%m%d_%H%M%S")

'20231015_230811'

In [8]:
# alldata[0]

In [9]:
df = pd.DataFrame(alldata)

file_to_save = os.path.join(b_folder ,'Total'+datetime.now().strftime("%Y%m%d_%H%M%S")+'.pickle')
df.to_pickle(file_to_save)

In [10]:
df.head(2)

,metadata,date,area,courseid,json,id,desc
0,"MGC004202309197H브라운패취(그린),조류(그린),볼마크(그린)_0",20230919,7H,MGC004,{'originalFileGrp': ['ta/nas/10_원시데이터(3사_분류대상)...,20230919114349_1267681724167_373693833056_4798...,"브라운패취(그린),조류(그린),볼마크(그린)_0"
1,"MGC004202309197H브라운패취(그린),조류(그린),볼마크(그린)_0",20230919,7H,MGC004,{'originalFileGrp': ['ta/nas/10_원시데이터(3사_분류대상)...,20230919114350_1267681720833_373693834444_4797...,"브라운패취(그린),조류(그린),볼마크(그린)_0"


In [116]:
df['path'] = df.json.map(lambda x: x.get('destFolder'))
df['NumLabel'] = df.json.map(lambda x: len(x.get('label')))
df['NumTBDLabel'] = df.json.map(lambda x: len([label_ for label_ in x.get('label') if label_.get('level1') == "TBD"]))


In [117]:
df.loc[:,'LabelType'] = 'NoLabel'
df.loc[df['NumLabel']>0,'LabelType'] = 'Label'
df.loc[df['NumTBDLabel']>0,'LabelType'] = 'TBD'

In [139]:
sum_df = df.groupby(['courseid','date','path','LabelType']).agg({'id':'count'}).reset_index().rename(columns = {'id':'file_cnt'})

In [140]:
sum_df = sum_df.pivot_table('file_cnt', ['courseid','date','path'], 'LabelType').fillna(0).reset_index()
sum_df['filepath'] = sum_df['courseid'] +'/' + sum_df['date'] +'/'+ 'filepath_'+sum_df['courseid'] + '_'+sum_df['date']+'.json'
sum_df['file_cnt'] = sum_df['Label'] + sum_df['NoLabel'] + sum_df['TBD']
sum_df['labeled_file_cnt'] = sum_df['Label']
sum_df['TBD_file_cnt'] = sum_df['TBD']
report_df = sum_df[['filepath','courseid','date','path','file_cnt', 'labeled_file_cnt', 'TBD_file_cnt']]

In [141]:
report_df.head(5)

LabelType,filepath,courseid,date,path,file_cnt,labeled_file_cnt,TBD_file_cnt
0,MGC001/20230811/filepath_MGC001_20230811.json,MGC001,20230811,"20230811/동해 4H/물리적피해,과습피해,플레이어답압,토양병,페어리링",130.0,130.0,0.0
1,MGC001/20230811/filepath_MGC001_20230811.json,MGC001,20230811,"20230811/동해 4H/물리적피해,음지피해,잡초,이종잔디,패취류,썸머패취",205.0,0.0,0.0
2,MGC001/20230811/filepath_MGC001_20230811.json,MGC001,20230811,"20230811/동해 4H/물리적피해,음지피해,켄터키,티,잡초,이종잔디,켄터키,패취...",174.0,1.0,0.0
3,MGC001/20230811/filepath_MGC001_20230811.json,MGC001,20230811,"20230811/동해 4H/물리적피해,플레이어답압",42.0,1.0,0.0
4,MGC001/20230811/filepath_MGC001_20230811.json,MGC001,20230811,20230811/동해 4H/미기재,136.0,0.0,0.0


In [147]:
for filepath_ in list(report_df.filepath.unique()):
  file2save = os.path.join(out_folder, 'filepath_'+filepath_[:6] + '_' + filepath_[7:15]+'.json')
  jsonFile = report_df.loc[report_df.filepath == filepath_,['path','file_cnt', 'labeled_file_cnt', 'TBD_file_cnt']].to_json(orient='records', force_ascii=False)
  
  with open(file2save, 'w', encoding="utf-8") as f:
    f.write(jsonFile)

  s3.put_object(
  Body=jsonFile,
  Bucket='ds-niaimageskor',
  Key= 'public/'+filepath_
  )

In [148]:
jsonFile

'[{"path":"20230907\\/1.힐코스_1H\\/그린칼라,파습,조류,러프 이종-벤트_0","file_cnt":282.0,"labeled_file_cnt":0.0,"TBD_file_cnt":0.0},{"path":"20230907\\/1.힐코스_1H\\/답압, 피슘피해지,썸머패치, 이종잔디-중지, 벤트,중지황화, 잡초-파대가리, 바랭이,벤트고사지_0","file_cnt":501.0,"labeled_file_cnt":0.0,"TBD_file_cnt":0.0},{"path":"20230907\\/1.힐코스_1H\\/답압, 피슘피해지,썸머패치, 이종잔디-중지, 벤트,중지황화, 잡초-파대가리, 바랭이,벤트고사지_1","file_cnt":501.0,"labeled_file_cnt":0.0,"TBD_file_cnt":0.0},{"path":"20230907\\/1.힐코스_1H\\/답압, 피슘피해지,썸머패치, 이종잔디-중지, 벤트,중지황화, 잡초-파대가리, 바랭이,벤트고사지_2","file_cnt":4.0,"labeled_file_cnt":0.0,"TBD_file_cnt":0.0},{"path":"20230907\\/2.힐코스_2H\\/잡초, 그린촬영_0","file_cnt":109.0,"labeled_file_cnt":0.0,"TBD_file_cnt":0.0},{"path":"20230907\\/2.힐코스_2H\\/잡초,그린촬영_0","file_cnt":223.0,"labeled_file_cnt":0.0,"TBD_file_cnt":0.0},{"path":"20230907\\/3.힐코스_6H\\/중지러프, 잡초-방동사니,매듭풀, 참수지.바퀴자국_0","file_cnt":366.0,"labeled_file_cnt":0.0,"TBD_file_cnt":0.0},{"path":"20230907\\/3.힐코스_6H\\/티노리, 경사면, 잡초,이끼_0","file_cnt":371.0,"labeled_file_cnt":107.0,"TBD_file_cnt":0.0},{"path

In [143]:
fileList = glob.glob(os.path.join(out_folder,'filepath*.json'), recursive=True)
fileList

['c:\\Users\\yc463\\Documents\\JamesLocal\\DS\\Drone\\dsmapv2\\src\\AWS\\filepathBackUp\\filepath_MGC001_20230811.json',
 'c:\\Users\\yc463\\Documents\\JamesLocal\\DS\\Drone\\dsmapv2\\src\\AWS\\filepathBackUp\\filepath_MGC001_20230816.json',
 'c:\\Users\\yc463\\Documents\\JamesLocal\\DS\\Drone\\dsmapv2\\src\\AWS\\filepathBackUp\\filepath_MGC001_20230908.json',
 'c:\\Users\\yc463\\Documents\\JamesLocal\\DS\\Drone\\dsmapv2\\src\\AWS\\filepathBackUp\\filepath_MGC002_20230823.json',
 'c:\\Users\\yc463\\Documents\\JamesLocal\\DS\\Drone\\dsmapv2\\src\\AWS\\filepathBackUp\\filepath_MGC002_20230907.json',
 'c:\\Users\\yc463\\Documents\\JamesLocal\\DS\\Drone\\dsmapv2\\src\\AWS\\filepathBackUp\\filepath_MGC002_20230919.json',
 'c:\\Users\\yc463\\Documents\\JamesLocal\\DS\\Drone\\dsmapv2\\src\\AWS\\filepathBackUp\\filepath_MGC002_20230927.json',
 'c:\\Users\\yc463\\Documents\\JamesLocal\\DS\\Drone\\dsmapv2\\src\\AWS\\filepathBackUp\\filepath_MGC003_20230909.json',
 'c:\\Users\\yc463\\Documents\\J

In [144]:
course_id = "MGC002"
target_date = '20230823'
file_ = [x for x in fileList if course_id in x and target_date in x][0]
file_

'c:\\Users\\yc463\\Documents\\JamesLocal\\DS\\Drone\\dsmapv2\\src\\AWS\\filepathBackUp\\filepath_MGC002_20230823.json'

In [145]:
filepath_  = list(report_df.filepath.unique())[3]
filepath_

'MGC002/20230823/filepath_MGC002_20230823.json'

In [146]:
s3 = boto3.client('s3')

with open(file_, "r", encoding='utf-8') as _json:
  pathInfo= json.load(_json)

s3.put_object(
  Body=json.dumps(pathInfo).encode('utf-8'),
  Bucket='ds-niaimageskor',
  Key= 'public/'+filepath_
  )



{'ResponseMetadata': {'RequestId': '98YGXDG8CZHKRMPR',
  'HostId': 'HJDIRCdTz2xDSUTGj6YK1iTofev9mGVIq1q+srxD0Z+6TrPt381HKWlI9ihD7N/90g8OcLTvtdg=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'HJDIRCdTz2xDSUTGj6YK1iTofev9mGVIq1q+srxD0Z+6TrPt381HKWlI9ihD7N/90g8OcLTvtdg=',
   'x-amz-request-id': '98YGXDG8CZHKRMPR',
   'date': 'Sat, 14 Oct 2023 11:42:48 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"a5f50d0edfa44f0f0d4cd1a7a17c5529"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 1},
 'ETag': '"a5f50d0edfa44f0f0d4cd1a7a17c5529"',
 'ServerSideEncryption': 'AES256'}

In [147]:
for index_, row_ in enumerate(alldata):
  row_['courseid'] = row_['metadata'][:6]


In [148]:
with table.batch_writer(overwrite_by_pkeys=['metadata', 'id']) as batch:
  for index_, row_ in enumerate(alldata):
    print(index_, row_['id'], end='\r')
    batch.put_item(
      Item= row_
    )

In [149]:
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError

courseid = "MGC004"
date = "20230919"


response = table.scan(
    # IndexName='id-courseid-index',
    FilterExpression=Attr('metadata').begins_with(courseid)&Attr('date').eq(date)
)

data = response['Items']

while 'LastEvaluatedKey' in response:
    response = table.scan(
      # IndexName='id-courseid-index',
      FilterExpression=Attr('metadata').begins_with(courseid)&Attr('date').eq(date),
      ExclusiveStartKey=response['LastEvaluatedKey'])
    data.extend(response['Items'])

In [150]:
len(data)

8635

In [153]:
courseid = "MGC004"
date = "20230919"


response = table.query(IndexName='courseid-id-index', KeyConditionExpression=Key('courseid').eq(courseid),  FilterExpression=Attr('date').eq(date))
data =  response['Items']

while 'LastEvaluatedKey' in response:
    response = table.query(IndexName='courseid-id-index', KeyConditionExpression=Key('courseid').eq(courseid),  ExclusiveStartKey=response['LastEvaluatedKey'] ,FilterExpression=Attr('date').eq(date) )    
    # response_ = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    data.extend(response['Items'])

In [154]:
len(data)

8635